In [1]:
import datasets

ds = datasets.load_dataset('joyheyueya/250428_sft', split='train')
ds

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 749
})

In [2]:
ds[0]

{'prompt': 'Your task is to identify and elaborate on an insight that only becomes apparent by combining information from both documents together—i.e., an insight that has high relevance when treating the documents jointly but low relevance if you were to consider each document alone. Write the insight as a standalone statement that does not reference the original texts or use terms like "paper" or "document". The insight should involve indirect connections or intermediate reasoning steps, such as linking cause and effect through a shared variable or mechanism. Here are some examples: Let A = magnesium — either dietary or internal to the body. Implicitly refers, inmost cases, to quantity of magnesium. Let M = migraine headache. Let L = one or more intermediate physiological links.  -> means \'can or might influence\'. == means \'equivalent in action\' or \'equivalent in mechanism\' depending on whether it connects two drugs or two diseases. Paper 1:\nStress and Type A behavior are asso

In [3]:
import os
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

def apply_chat_template(example):
    example['prompt'] = example['prompt'].replace("Let's think step by step.", ". \n\nLet's think step by step.")
    chat = [{"role": "user", "content": example['prompt']}]
    example['query'] = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    return example

ds = ds.map(apply_chat_template, batched=False, num_proc=os.cpu_count(), remove_columns=['prompt'])
print(ds[0]['query'])


Map (num_proc=24):   0%|          | 0/749 [00:00<?, ? examples/s]

<|im_start|>user
Your task is to identify and elaborate on an insight that only becomes apparent by combining information from both documents together—i.e., an insight that has high relevance when treating the documents jointly but low relevance if you were to consider each document alone. Write the insight as a standalone statement that does not reference the original texts or use terms like "paper" or "document". The insight should involve indirect connections or intermediate reasoning steps, such as linking cause and effect through a shared variable or mechanism. Here are some examples: Let A = magnesium — either dietary or internal to the body. Implicitly refers, inmost cases, to quantity of magnesium. Let M = migraine headache. Let L = one or more intermediate physiological links.  -> means 'can or might influence'. == means 'equivalent in action' or 'equivalent in mechanism' depending on whether it connects two drugs or two diseases. Paper 1:
Stress and Type A behavior are associ

In [4]:
ds = ds.shuffle(seed=42)
num_test = 49
num_train = len(ds) - num_test

train_ds = ds.select(range(num_train))
test_ds = ds.select(range(num_train, len(ds)))

train_ds

Dataset({
    features: ['completion', 'query'],
    num_rows: 700
})

In [5]:
output_dir = '/home/anikait.singh/rl_behaviors_verl_stable/data_insights_sft'
train_ds.to_parquet(os.path.join(output_dir, 'train.parquet'))
test_ds.to_parquet(os.path.join(output_dir, 'test.parquet'))

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

310271